In [1]:
import PIL
import gc
import torch
import torchvision
import os

import numpy as np
import matplotlib.pyplot as plt  
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets, transforms
from torch.utils.data import Subset
from IPython.core.display import display, HTML
from numpy.random import RandomState
from wide_resnet import WideResNet
from auto_augment import AutoAugment, Cutout
from efficientnet_pytorch import EfficientNet
from cifar_loader import SmallSampleController
import torchvision.models as models


# display(HTML("<style>.container { width:40% !important; }</style>"))


In [2]:

def getAcc(preds,targets):
    return np.sum([1 if preds[i] == targets[i] else 0 for i in range(len(preds))])/len(preds)

def train(model, device, train_loader, optimizer, epoch, display=True):
    """
    Summary: Implements the training procedure for a given model
    == params ==
    model: the model to test
    device: cuda or cpu 
    optimizer: the optimizer for our training
    train_loader: dataloader for our train data
    display: output flag
    == output ==
    the mean train loss, the train accuracy
    """
    
    lossTracker = []
    
    targets=[]
    preds=[]
    
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        lossTracker.append(loss.detach())
        with torch.no_grad():
            pred = torch.argmax(output,1).cpu().numpy()

            preds.extend(pred)
            targets.extend(target.cpu().numpy())
        
    lossTracker = [x.item() for x in lossTracker]
    meanLoss = np.mean(lossTracker)
    accuracy = getAcc(preds,targets)
    if display:
        print('Train Epoch: {} [acc: {:.0f}%]\tLoss: {:.6f}'.format(
          epoch, 100. * accuracy, meanLoss))
        
    return accuracy, meanLoss



def test(model, device, test_loader,verbose=True):
    """
    Summary: Implements the testing procedure for a given model
    == params ==
    model: the model to test
    device: cuda or cpu 
    test_loader: dataloader for our test data
    verbose: output flag
    == output ==
    the mean test loss, the test accuracy
    """
    
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            

    meanLoss = test_loss / len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    
    if verbose: print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        mean_test_loss, correct, len(test_loader.dataset),
        accuracy))
        
    return accuracy, meanLoss


def checkTest(model,device,valSets,valTracker,latexTracker,epoch,
              model_name,optim_name,lr,totalTestSamples,seed,verbose=True):
    """
    Summary: checks the test accuracy, prints, and saves statistics
    """
    tempAcc = []
    tempLoss = []
    for val_loader in valSets:
        acc,loss = test(model, device, val_loader,verbose = False)
        tempAcc.append(acc)
        tempLoss.append(loss)
        
    meanAcc = np.mean(tempAcc)
    stdAcc = np.std(tempAcc)
    
    meanLoss = np.mean(tempLoss)
    if verbose:
        print('[Trained for {} epochs and tested on {} sets of 2000 images]\
        Avg Acc: {:.2f} +- {:.2f} , Avg Loss: {:.2f}'.format(
            epoch,VALIDATION_SET_NUM,meanAcc,stdAcc,meanLoss))
        
        
    tableRow = getLatexRow(architecture=model_name,epoch=epoch,accuracy=meanAcc,optim=optim_name,
                           lr=lr,totalTestSamples=totalTestSamples,dataAug="Nothing",
                           seed=seed,title=False)
    
    latexTracker.append(tableRow)
        
    valTracker["allLoss"].extend(tempLoss)
    valTracker["allAcc"].extend(tempAcc)
    valTracker["meanLoss"].append(meanLoss)
    valTracker["meanAcc"].append(meanAcc)
    valTracker["stdAcc"].append(stdAcc)





In [3]:
def getLatexRow(architecture,epoch,accuracy,optim,lr,
                totalTestSamples,dataAug,seed,title=False):
    """
    Summary: generates one row of latex for a results table
    """
    categories = ["Model","Epoch","Accuracy","Optimizer","lr","Test Sample Num",
                  "data augmentation","seed"]
    row = [str(architecture),str(epoch),str(round(accuracy,3)),str(optim),
           str(lr),str(totalTestSamples),str(dataAug),str(seed)]
    
    if title:
        c = "&".join(categories)
        r = "&".join(row)
        return "{}\\\\\n{}\\\\".format(c,r)
    else:
        r = "&".join(row)
        return "{}\\\\".format(r)
    
    
def plot(xlist,ylist,xlab,ylab,title,color,label,savedir=".",save=False):
    """
    Summary: plots the given list of numbers against its idices and 
    allows for high resolution saving
    """
    fig = plt.figure()
    plt.title(title)
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.plot(xlist,ylist,color=color,marker=".",label=label)
    plt.legend()
    
    if save:
        if not os.path.isdir(savedir):
            os.mkdir(savedir)
        filepath = os.path.join(savedir,"{}".format(title))
        plt.savefig(filepath+".pdf")
        os.system("pdftoppm -png -r 300 {}.pdf {}.png".format(filepath,filepath))
        
    plt.show()
    
    

In [4]:
def getModel(model_name):
    if "wide" in model_name.lower():
        return WideResNet(28, 10, num_classes=10)
    elif "efficient" in model_name.lower():
        return EfficientNet.from_pretrained(model_name,num_classes = 10) # change to not be pretrained
    elif "vgg16" in model_name.lower():
        model = models.vgg16(pretrained=True)
#         model.classifier[6] = nn.Linear(4096, 10)
        return model
    elif "alexnet" in model_name.lower():
        model = models.alexnet(pretrained=True)
#         model.classifier = nn.Linear(256 * 6 * 6, 10)
        return model
    elif "resnet18" in model_name.lower():
        model = models.resnet18(pretrained=True)
#         model.fc.out_features = 10
        return model
    
    
def getOptimizer128(optimizer_name,parameters):
    if "sgd" in  optimizer_name.lower():
        LR = 0.1
        optim = torch.optim.SGD(parameters, 
                                  lr=LR, momentum=0.9,
                                  weight_decay=0.0005)
        return optim, LR
    elif "adam" in optimizer_name.lower():
        LR = 0.00005
        optim = torch.optim.Adam(parameters, 
                              lr=LR, weight_decay=0)
        return optim, LR
    elif "rmsprop" in optimizer_name.lower():
        LR = 0.01
        optim = torch.optim.RMSprop(params, lr=0.256, alpha=0.99, eps=1e-08, 
                            weight_decay=0.00005, momentum=0.00, centered=False)
        return optim, LR
    
        
    

In [5]:
class featureExtractor(nn.Module):
    def __init__(self,efficientNet):
        super().__init__()
        self.eNet = efficientNet 
        self._avg_pooling = nn.AdaptiveAvgPool2d(1)
        self._dropout = nn.Dropout(self.eNet._global_params.dropout_rate)
        self._fc = nn.Linear(2560,10)
        
        
    def forward(self,inputs):
        x = self.eNet.extract_features(inputs)
        x = x.flatten(start_dim=1)
        x = self._dropout(x)
        x = self._fc(x)
        return x

In [ ]:
torch.cuda.empty_cache()
gc.collect()

#     resnet18 = models.resnet18()
#     alexnet = models.alexnet()
#     vgg16 = models.vgg16()
#     squeezenet = models.squeezenet1_0()
#     densenet = models.densenet161()
#     inception = models.inception_v3()
#     googlenet = models.googlenet()
#     shufflenet = models.shufflenet_v2_x1_0()
#     mobilenet = models.mobilenet_v2()
#     resnext50_32x4d = models.resnext50_32x4d()
#     wide_resnet50_2 = models.wide_resnet50_2()
#     mnasnet = models.mnasnet1_0()

OPTIM = "rmsprop"
MODEL = "efficientnet-b2"
EPOCH_NUM = 100000
TRAIN_SAMPLE_NUM = 100
VAL_SAMPLE_NUM = 2000
BATCH_SIZE = 128
VALIDATION_SET_NUM = 5
AUGMENT = True
VAL_DISPLAY_DIVISOR =25
CIFAR_TRAIN = True

#cifar-10:
#mean = (0.4914, 0.4822, 0.4465)
#std = (0.247, 0.243, 0.261)

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
if AUGMENT:
    dataAugmentation = [ 
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        AutoAugment(),
        Cutout()
    ]
    augment = "Crop,Flip,AutoAugment,Cutout"
else: 
    dataAugmentation = []
    augment = "Nothing"


# We resize images to allow using imagenet pre-trained models, is there a better way?
resize = transforms.Resize(160) 

transform_train = transforms.Compose(dataAugmentation + [transforms.ToTensor(), normalize]) 
transform_val = transforms.Compose([transforms.ToTensor(), normalize]) #careful to keep this one same

cifar_train = datasets.CIFAR10(root='.',train=CIFAR_TRAIN, transform=transform_train, download=True)
cifar_val = datasets.CIFAR10(root='.',train=CIFAR_TRAIN, transform=transform_val, download=True)

ss = SmallSampleController(numClasses=10,trainSampleNum=TRAIN_SAMPLE_NUM, # abstract the data-loading procedure
                           valSampleNum=VAL_SAMPLE_NUM, batchSize=BATCH_SIZE, 
                           multiplier=VALIDATION_SET_NUM, trainDataset=cifar_train, 
                           valDataset=cifar_val)
    
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_data, valSets, seed = ss.generateNewSet(device,valMultiplier = VALIDATION_SET_NUM) #Sample from datasets



eNet = getModel(MODEL).cuda()

# for param in eNet.parameters():
#     param.requires_grad = False
    
# model = featureExtractor(eNet).cuda()
model = eNet
    
optimizer,LR = getOptimizer128(OPTIM,model._fc.parameters())

print(model.parameters)

print(' => Total trainable parameters: %.2fM' % (sum(p.numel() for p in model.parameters()) / 1000000.0))        

trainTracker = {"meanLoss":[],"accuracy":[]}
valTracker = {"allLoss":[],"allAcc":[],"meanLoss":[],"meanAcc":[],"stdAcc":[]}
latexTracker = []

print("Begin Train for {} epochs".format(EPOCH_NUM))
for epoch in range(EPOCH_NUM):
    acc, loss = train(model, device, train_data[0], optimizer, epoch+1, display=True)
    trainTracker["meanLoss"].append(loss)
    trainTracker["accuracy"].append(acc)
    
    if (epoch+1) % VAL_DISPLAY_DIVISOR == 0:
        checkTest(model,device,valSets,valTracker,latexTracker,epoch+1,
              model_name=MODEL,optim_name=OPTIM,lr=LR,totalTestSamples=VAL_SAMPLE_NUM*VALIDATION_SET_NUM,
                  seed=seed,verbose=True)
        
          
        
        

Files already downloaded and verified
Files already downloaded and verified
Generated new permutation of the CIFAR train dataset with                 seed:1619993594, train sample num: 100, test sample num: 2000
Loaded pretrained weights for efficientnet-b2
<bound method Module.parameters of EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, k

Train Epoch: 1 [acc: 12%]	Loss: 2.394218
Train Epoch: 2 [acc: 12%]	Loss: 2.385690
Train Epoch: 3 [acc: 19%]	Loss: 2.301321
Train Epoch: 4 [acc: 10%]	Loss: 2.292584
Train Epoch: 5 [acc: 23%]	Loss: 2.220886
Train Epoch: 6 [acc: 25%]	Loss: 2.212301
Train Epoch: 7 [acc: 22%]	Loss: 2.181154
Train Epoch: 8 [acc: 27%]	Loss: 2.300593
Train Epoch: 9 [acc: 27%]	Loss: 2.258864
Train Epoch: 10 [acc: 25%]	Loss: 2.210883
Train Epoch: 11 [acc: 21%]	Loss: 2.815822
Train Epoch: 12 [acc: 29%]	Loss: 2.432531
Train Epoch: 13 [acc: 26%]	Loss: 2.317907
Train Epoch: 14 [acc: 28%]	Loss: 2.853905
Train Epoch: 15 [acc: 24%]	Loss: 3.002356
Train Epoch: 16 [acc: 26%]	Loss: 2.643939
Train Epoch: 17 [acc: 30%]	Loss: 2.394347
Train Epoch: 18 [acc: 21%]	Loss: 3.229366
Train Epoch: 19 [acc: 34%]	Loss: 2.715906
Train Epoch: 20 [acc: 22%]	Loss: 3.206520
Train Epoch: 21 [acc: 27%]	Loss: 3.019003
Train Epoch: 22 [acc: 26%]	Loss: 3.069120
Train Epoch: 23 [acc: 25%]	Loss: 3.009127
Train Epoch: 24 [acc: 28%]	Loss: 2.867465
T

/home/benjamin/venv/torch11/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


[Trained for 25 epochs and tested on 5 sets of 2000 images]        Avg Acc: 10.01 +- 0.10 , Avg Loss: 2.77
Train Epoch: 26 [acc: 22%]	Loss: 3.208790
Train Epoch: 27 [acc: 27%]	Loss: 3.222914
Train Epoch: 28 [acc: 28%]	Loss: 3.604144
Train Epoch: 29 [acc: 27%]	Loss: 3.571048
Train Epoch: 30 [acc: 22%]	Loss: 3.463038
Train Epoch: 31 [acc: 31%]	Loss: 3.074571
Train Epoch: 32 [acc: 35%]	Loss: 2.936583
Train Epoch: 33 [acc: 22%]	Loss: 3.945976
Train Epoch: 34 [acc: 30%]	Loss: 3.106127
Train Epoch: 35 [acc: 30%]	Loss: 3.321368
Train Epoch: 36 [acc: 27%]	Loss: 3.783017
Train Epoch: 37 [acc: 26%]	Loss: 3.514533
Train Epoch: 38 [acc: 25%]	Loss: 3.993915
Train Epoch: 39 [acc: 28%]	Loss: 4.051842
Train Epoch: 40 [acc: 22%]	Loss: 3.946180
Train Epoch: 41 [acc: 34%]	Loss: 3.761600
Train Epoch: 42 [acc: 34%]	Loss: 3.129751
Train Epoch: 43 [acc: 21%]	Loss: 4.319491
Train Epoch: 44 [acc: 43%]	Loss: 2.833761
Train Epoch: 45 [acc: 29%]	Loss: 3.866552
Train Epoch: 46 [acc: 31%]	Loss: 3.953522
Train Epoch

[Trained for 200 epochs and tested on 5 sets of 2000 images]        Avg Acc: 15.98 +- 0.60 , Avg Loss: 5.27
Train Epoch: 201 [acc: 35%]	Loss: 4.902340
Train Epoch: 202 [acc: 30%]	Loss: 6.456171
Train Epoch: 203 [acc: 28%]	Loss: 5.321291
Train Epoch: 204 [acc: 34%]	Loss: 5.030759
Train Epoch: 205 [acc: 34%]	Loss: 5.365110
Train Epoch: 206 [acc: 32%]	Loss: 4.791469
Train Epoch: 207 [acc: 23%]	Loss: 5.830007
Train Epoch: 208 [acc: 30%]	Loss: 5.588132
Train Epoch: 209 [acc: 34%]	Loss: 4.534493
Train Epoch: 210 [acc: 22%]	Loss: 6.431704
Train Epoch: 211 [acc: 29%]	Loss: 4.886855
Train Epoch: 212 [acc: 30%]	Loss: 4.971320
Train Epoch: 213 [acc: 27%]	Loss: 5.743999
Train Epoch: 214 [acc: 31%]	Loss: 4.744389
Train Epoch: 215 [acc: 25%]	Loss: 5.719297
Train Epoch: 216 [acc: 34%]	Loss: 4.869385
Train Epoch: 217 [acc: 30%]	Loss: 5.091959
Train Epoch: 218 [acc: 29%]	Loss: 4.945020
Train Epoch: 219 [acc: 33%]	Loss: 5.687912
Train Epoch: 220 [acc: 36%]	Loss: 4.612198
Train Epoch: 221 [acc: 32%]	Loss

...

Train Epoch: 1403 [acc: 33%]	Loss: 5.719269
Train Epoch: 1404 [acc: 25%]	Loss: 6.612069
Train Epoch: 1405 [acc: 26%]	Loss: 5.757385
Train Epoch: 1406 [acc: 27%]	Loss: 5.836544
Train Epoch: 1407 [acc: 27%]	Loss: 5.927109
Train Epoch: 1408 [acc: 30%]	Loss: 6.251940
Train Epoch: 1409 [acc: 27%]	Loss: 5.697063
Train Epoch: 1410 [acc: 39%]	Loss: 4.480721
Train Epoch: 1411 [acc: 28%]	Loss: 5.808690
Train Epoch: 1412 [acc: 25%]	Loss: 5.943625
Train Epoch: 1413 [acc: 32%]	Loss: 5.178919
Train Epoch: 1414 [acc: 32%]	Loss: 5.466706
Train Epoch: 1415 [acc: 30%]	Loss: 6.351055
Train Epoch: 1416 [acc: 26%]	Loss: 6.153141
Train Epoch: 1417 [acc: 39%]	Loss: 4.812960
Train Epoch: 1418 [acc: 30%]	Loss: 5.907178
Train Epoch: 1419 [acc: 34%]	Loss: 5.558563
Train Epoch: 1420 [acc: 37%]	Loss: 5.302495
Train Epoch: 1421 [acc: 30%]	Loss: 6.222645
Train Epoch: 1422 [acc: 29%]	Loss: 6.443831
Train Epoch: 1423 [acc: 23%]	Loss: 6.493947
Train Epoch: 1424 [acc: 30%]	Loss: 4.764655
Train Epoch: 1425 [acc: 29%]	Los

Train Epoch: 1575 [acc: 27%]	Loss: 6.632133
[Trained for 1575 epochs and tested on 5 sets of 2000 images]        Avg Acc: 15.73 +- 0.53 , Avg Loss: 7.97
Train Epoch: 1576 [acc: 24%]	Loss: 5.399893
Train Epoch: 1577 [acc: 33%]	Loss: 5.516624
Train Epoch: 1578 [acc: 34%]	Loss: 5.200974
Train Epoch: 1579 [acc: 39%]	Loss: 5.214938
Train Epoch: 1580 [acc: 33%]	Loss: 5.493948
Train Epoch: 1581 [acc: 28%]	Loss: 6.102060
Train Epoch: 1582 [acc: 39%]	Loss: 5.422990
Train Epoch: 1583 [acc: 24%]	Loss: 5.372546
Train Epoch: 1584 [acc: 32%]	Loss: 5.196605
Train Epoch: 1585 [acc: 29%]	Loss: 5.849141
Train Epoch: 1586 [acc: 27%]	Loss: 5.403432
Train Epoch: 1587 [acc: 24%]	Loss: 5.152740
Train Epoch: 1588 [acc: 36%]	Loss: 5.048390
Train Epoch: 1589 [acc: 27%]	Loss: 5.794664
Train Epoch: 1590 [acc: 34%]	Loss: 5.413836
Train Epoch: 1591 [acc: 29%]	Loss: 6.181556
Train Epoch: 1592 [acc: 26%]	Loss: 5.472163
Train Epoch: 1593 [acc: 34%]	Loss: 5.555897
Train Epoch: 1594 [acc: 36%]	Loss: 4.835930
Train Epoch

Train Epoch: 1744 [acc: 34%]	Loss: 5.191627
Train Epoch: 1745 [acc: 27%]	Loss: 6.296119
Train Epoch: 1746 [acc: 34%]	Loss: 6.177954
Train Epoch: 1747 [acc: 31%]	Loss: 5.091797
Train Epoch: 1748 [acc: 27%]	Loss: 6.571234
Train Epoch: 1749 [acc: 22%]	Loss: 6.859026
Train Epoch: 1750 [acc: 26%]	Loss: 5.917472
[Trained for 1750 epochs and tested on 5 sets of 2000 images]        Avg Acc: 16.30 +- 0.69 , Avg Loss: 8.05
Train Epoch: 1751 [acc: 31%]	Loss: 5.359730
Train Epoch: 1752 [acc: 29%]	Loss: 5.505345
Train Epoch: 1753 [acc: 27%]	Loss: 6.412131
Train Epoch: 1754 [acc: 33%]	Loss: 5.113166
Train Epoch: 1755 [acc: 31%]	Loss: 6.007956
Train Epoch: 1756 [acc: 27%]	Loss: 5.723130
Train Epoch: 1757 [acc: 30%]	Loss: 6.620576
Train Epoch: 1758 [acc: 28%]	Loss: 6.232605
Train Epoch: 1759 [acc: 23%]	Loss: 6.849936
Train Epoch: 1760 [acc: 33%]	Loss: 5.330819
Train Epoch: 1761 [acc: 20%]	Loss: 6.500650
Train Epoch: 1762 [acc: 29%]	Loss: 5.776295
Train Epoch: 1763 [acc: 29%]	Loss: 4.590209
Train Epoch

Train Epoch: 1913 [acc: 35%]	Loss: 5.855463
Train Epoch: 1914 [acc: 29%]	Loss: 7.066701
Train Epoch: 1915 [acc: 26%]	Loss: 6.308536
Train Epoch: 1916 [acc: 31%]	Loss: 5.559658
Train Epoch: 1917 [acc: 32%]	Loss: 5.013192
Train Epoch: 1918 [acc: 25%]	Loss: 6.203205
Train Epoch: 1919 [acc: 28%]	Loss: 6.220907
Train Epoch: 1920 [acc: 33%]	Loss: 5.895517
Train Epoch: 1921 [acc: 42%]	Loss: 4.601152
Train Epoch: 1922 [acc: 38%]	Loss: 4.828623
Train Epoch: 1923 [acc: 28%]	Loss: 4.516589
Train Epoch: 1924 [acc: 33%]	Loss: 4.602806
Train Epoch: 1925 [acc: 21%]	Loss: 6.525775
[Trained for 1925 epochs and tested on 5 sets of 2000 images]        Avg Acc: 15.54 +- 0.90 , Avg Loss: 8.04
Train Epoch: 1926 [acc: 33%]	Loss: 5.510846
Train Epoch: 1927 [acc: 29%]	Loss: 6.193460
Train Epoch: 1928 [acc: 35%]	Loss: 6.041289
Train Epoch: 1929 [acc: 33%]	Loss: 5.550560
Train Epoch: 1930 [acc: 25%]	Loss: 6.310377
Train Epoch: 1931 [acc: 37%]	Loss: 5.538084
Train Epoch: 1932 [acc: 28%]	Loss: 6.530467
Train Epoch

Train Epoch: 2082 [acc: 37%]	Loss: 4.135371
Train Epoch: 2083 [acc: 35%]	Loss: 5.366210
Train Epoch: 2084 [acc: 30%]	Loss: 5.751951
Train Epoch: 2085 [acc: 30%]	Loss: 6.191255
Train Epoch: 2086 [acc: 38%]	Loss: 5.003034
Train Epoch: 2087 [acc: 31%]	Loss: 6.419865
Train Epoch: 2088 [acc: 30%]	Loss: 5.257380
Train Epoch: 2089 [acc: 30%]	Loss: 5.219092
Train Epoch: 2090 [acc: 29%]	Loss: 5.340225
Train Epoch: 2091 [acc: 26%]	Loss: 5.734572
Train Epoch: 2092 [acc: 36%]	Loss: 4.636786
Train Epoch: 2093 [acc: 33%]	Loss: 4.777450
Train Epoch: 2094 [acc: 31%]	Loss: 5.444011
Train Epoch: 2095 [acc: 29%]	Loss: 5.352147
Train Epoch: 2096 [acc: 24%]	Loss: 6.865884
Train Epoch: 2097 [acc: 30%]	Loss: 5.348666
Train Epoch: 2098 [acc: 30%]	Loss: 5.644678
Train Epoch: 2099 [acc: 35%]	Loss: 5.117139
Train Epoch: 2100 [acc: 34%]	Loss: 5.369485
[Trained for 2100 epochs and tested on 5 sets of 2000 images]        Avg Acc: 15.68 +- 1.03 , Avg Loss: 8.21
Train Epoch: 2101 [acc: 30%]	Loss: 6.410943
Train Epoch

In [ ]:
dirname = latexTracker[-1][:-2] 

def writeTex(latexTracker,dirname):
    if not os.path.isdir(dirname):
        os.mkdir(dirname)
        
    f= open(os.path.join(dirname,"latexTable.txt"),"w")
    for x in latexTracker:
        f.write(x)
    f.close()

writeTex(latexTracker,dirname)

for x in latexTracker:
    print(x)

In [ ]:

epochList = [x+1 for x in range(len(trainTracker["meanLoss"]))]

plot(xlist=epochList,ylist=trainTracker["meanLoss"],xlab="Mean Train Loss",
    ylab="Epochs",title="Mean Train Loss over Epochs",
    color="#243A92",label="mean train loss",savedir=dirname,save=True)

plot(xlist=epochList,ylist=trainTracker["accuracy"],xlab="Train Accuracy",
    ylab="Epochs",title="Train Accuracy Over Epochs",
    color="#34267E",label="Train Accuracy",savedir=dirname,save=True)

In [ ]:

epochList = [VAL_DISPLAY_DIVISOR*(x+1) for x in range(len(valTracker["meanLoss"]))]

plot(xlist=epochList,ylist=valTracker["meanLoss"],xlab="Epochs",
    ylab="Mean Val Loss",title="Mean Val Loss over Epochs",
    color="#243A92",label="mean val loss",savedir=dirname,save=True)

plot(xlist=epochList,ylist=valTracker["meanAcc"],xlab="Epochs",
    ylab="Val Accuracy",title="Val Accuracy Over Epochs",
    color="#34267E",label="Val Accuracy",savedir=dirname,save=True)

plot(xlist=epochList,ylist=valTracker["stdAcc"],xlab="Epochs",
    ylab="Val Accuracy Standard Deviation",title="Val Accuracy Standard Deviation Over Epochs",
    color="#34267E",label="Val Accuracy SD",savedir=dirname,save=True)


valSetEvalCount = VAL_DISPLAY_DIVISOR * EPOCH_NUM * VALIDATION_SET_NUM
epochList = [VAL_DISPLAY_DIVISOR*(x+1) for x in range(len(valTracker["meanLoss"]))\
             for y in range(VALIDATION_SET_NUM)]


plot(xlist=epochList,ylist=valTracker["allLoss"],xlab="Val Set Evaluations",
    ylab="Val Loss",title="Val loss over val set evaluations ({} \
every {} epochs)".format(VALIDATION_SET_NUM,VAL_DISPLAY_DIVISOR),
    color="#34267E",label="Val Loss",savedir=dirname,save=True)

plot(xlist=epochList,ylist=valTracker["allAcc"],xlab="Val Set Evaluations",
    ylab="Val Accuracy",title="Val loss over val set evaluations ({} \
every {} epochs) ".format(VALIDATION_SET_NUM,VAL_DISPLAY_DIVISOR),
    color="#34267E",label="Val Accuracy",savedir=dirname,save=True)